## Rationale for the Final Notebook Name

This name captures all the successful elements and strategic decisions made during your optimization process:

* **`HTMP`**: The required acronym for the **Hull Tactical Market Prediction** competition.
* **`LGBM`**: Indicates the high-performing **LightGBM** algorithm, which proved superior to hybrid models.
* **`Sharpe_Max`**: Highlights the core objective of maximizing the **volatility-adjusted Sharpe Ratio** through the carefully chosen $0.55$ allocation factor.
* **`WFO`**: Acknowledges the underlying **Walk Forward Optimization principle** you adhered to: using the most recent 6,000 days of data and tuning for generalization, which is the necessary philosophy for time-series forecasting.
* **`V21`**: Denotes the **final, stable version** that incorporates all successful hyperparameter tuning for safety ($\text{n\_estimators}=150$) and performance ($\text{LR}=0.07, \text{MD}=9$).

A single, simple mathematical formula cannot represent the time required to train your LightGBM model. Instead, it's determined by a practical rule-of-thumb involving the model's complexity and your hardware's speed.

In the context of our Kaggle submission, the critical "formula" must follow is the **Training Time vs. Estimator Count Rule**:

## ⏰ Time Optimization Formula

Training time for boosting models like LightGBM scales roughly linearly with the number of estimators ($\text{n\_estimators}$) and the amount of data.

$$\text{Time}_{\text{Training}} \approx k \times \text{n\_estimators} \times N_{\text{samples}}$$

Where:
* **$\text{Time}_{\text{Training}}$:** The time taken for Phase I (must be $\mathbf{\leq 900 \text{ seconds}}$).
* **$k$**: A constant related to your hardware speed and the complexity of your features and depth.
* **$\text{n\_estimators}$**: The number of trees (your tuning parameter).
* **$N_{\text{samples}}$**: The number of training rows (fixed at 6,000 in your code).

### Your Safety Rule

Based on your executions, the constraint is:

$$\text{n\_estimators} \le 300$$

* **$\text{n\_estimators} = 300$**: Consumed $\approx 900$ seconds, putting you right at the edge of the time limit.
* **$\text{n\_estimators} = 990$**: Caused the time limit to be exceeded, leading to submission failure.

To ensure a valid submission, you should stick to $\text{n\_estimators} = 300$ as the maximum stable value.

Not exactly. The model was trained on $\mathbf{6,000}$ rows of data, which corresponds to approximately $\mathbf{16.43}$ calendar years, but represents close to **24 years of trading data**.

Here's the breakdown of why the larger number of years is more relevant for the trading competition:

---

## 📅 Trading vs. Calendar Years

When modelling financial data, the number of **trading days** is the standard measure, as it represents the number of independent data points (daily decisions) the model learns from.

* **Calendar Years:** $\frac{6,000 \text{ days}}{365.25 \text{ days/year}} \approx \mathbf{16.43 \text{ calendar years}}$
* **Trading Years:** $\frac{6,000 \text{ days}}{252 \text{ trading days/year}} \approx \mathbf{23.81 \text{ trading years}}$

The `df_train.tail(6000)` command selects the last 6,000 **trading days** present in the file. Therefore, the LightGBM model was effectively trained on a dataset spanning nearly **24 years of market history**.

This large window (24 years) is used to capture sufficient **market regime diversity** (including multiple economic cycles and recessions) to ensure the model generalizes well to unseen data during the forecasting phase.

Complete final solution, which is split into two cells: the **Exploratory Data Analysis (EDA) Cell** and the **Main Code Cell (LGBM-Dominant Ensemble)**. This structure maximizes code stability and time efficiency for your final submission.

---

## 1. Code Cell 1: Exploratory Data Analysis (EDA)

This cell performs initial data investigation and generates visualizations critical for understanding the nature of the target variable ($\text{market\_forward\_excess\_returns}$).

* **Purpose:** To visually confirm that the data is prepared correctly, identify key distributional properties, and spot highly correlated features before training the model.
* **Charts Generated:**
    1.  **Distribution of Returns:** A histogram with a Kernel Density Estimate ($\text{KDE}$) shows how often different return values occur. In finance, this typically confirms that returns are centered near zero but have "fat tails," indicating rare, extreme events (volatility).
    2.  **Time Series of Returns:** Plots the last 2000 days of returns against time. This helps identify market regimes, trends, and periods of high/low volatility clustering, which is essential context for the $\text{LGBM}$ model.
    3.  **Top Correlated Features:** A bar plot shows the top 10 features with the highest **absolute Spearman correlation** to the target. This identifies features most linearly and non-linearly related to the prediction goal, informing why the $\text{LGBM}$ model might find a strong signal.

---

## 2. Code Cell 2: Main Model Training and Inference

This cell contains the **LGBM-Dominant Ensemble**, which is the final, time-optimized prediction system.

### Phase I: Model Training (LGBM)

The goal here is to train the most stable model possible within the strict 900-second time limit.

* **Data Prep and Feature Set (Lines 63-75):** The script loads data, identifies $\mathbf{86}$ available features, converts columns, and focuses training on the most recent $\mathbf{6,000}$ rows for relevance.
* **Time Optimization (WFO Principle) (Lines 80-92):** The hyperparameters are manually tuned for generalization, mimicking the stability required by Walk Forward Optimization (WFO), which is crucial for a financial model:
    * **$\text{n\_estimators}=300$**: This is the **maximum safe value** that fits within the $\approx 900$-second time limit.
    * **$\text{learning\_rate}=0.03$ and $\text{max\_depth}=7$**: These conservative settings prevent the model from overfitting the training data's noise, which is necessary to achieve a high score on unseen test data.
* **Metric Report (Lines 95-99):** The script reports its training success:
    * $\text{Training R}^2$: **$0.5146$**. The $\text{LGBM}$ successfully explains over half the variance in the training data, confirming its strong predictive power.
* **Asset Management (Lines 101-104):** The trained $\text{LGBM}$ model is saved to disk. All logic related to the unstable $\text{CNN-LSTM}$ model is deliberately excluded, ensuring the script avoids the fatal $\text{IO}$ error during inference.

### Phase II: Submission Inference (The Prediction Loop)

The `predict(test: pl.DataFrame)` function executes every time the competition server requests a prediction.

| Code Section | Model Component | Weights | Rationale |
| :--- | :--- | :--- | :--- |
| **Model Loading** | **LGBM** | N/A | Uses a **lazy loading** block to load the $\text{LGBM}$ model *only* on the first call, avoiding the startup timeout risk. |
| **Component A** | $\text{LGBM Prediction}$ | $\mathbf{0.8}$ | The primary prediction is generated by the highly optimized $\text{LGBM}$ model, based on your historical best scores. |
| **Component C** | $\text{Linear Proxy (M1)}$ | $\mathbf{0.2}$ | A simple prediction based on the $\text{M1}$ momentum feature provides a stable baseline floor to the ensemble. |
| **Blending** | $\text{Raw Prediction}$ | $1.0$ | The predictions are summed by their weights. The $\text{LSTM}$ component's weight is $\mathbf{0.0}$ (disabled). |
| **Final Allocation** | $\text{Scaling/Clipping}$ | $\mathbf{0.55}$ | The raw prediction is multiplied by $\mathbf{0.55}$. This **Sharpe-optimizing scaling factor** controls the portfolio's exposure to volatility (risk) while maximizing the potential for a high return. The output is clipped to the competition's range of $[0.0, 2.0]$. |

In [ ]:
# --- EDA CELL: Exploratory Data Analysis for Financial Time Series ---
import matplotlib.pyplot as plt
import seaborn as sns 
from pathlib import Path
import warnings
import pandas as pd # Ensure pandas is imported if not already

# Suppress pandas warnings in this block
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

KAGGLE_INPUT_PATH = '/kaggle/input/hull-tactical-market-prediction/'
TARGET_COL = 'market_forward_excess_returns'

print("--- EDA PHASE: Analyzing Market Returns ---")
try:
    df_eda = pd.read_csv(Path(KAGGLE_INPUT_PATH) / 'train.csv')
    
    # 1. Convert Target to Numeric
    df_eda[TARGET_COL] = pd.to_numeric(df_eda[TARGET_COL], errors='coerce')
    
    # --- CHART 1: Distribution of Returns ---
    # Shows how centred returns are around zero (Volatility vs. Drift)
    plt.figure(figsize=(10, 6))
    sns.histplot(df_eda[TARGET_COL].dropna(), kde=True, bins=100)
    plt.title('1. Distribution of Market Forward Excess Returns (Target)')
    plt.show()

    # --- CHART 2: Time Series of Returns ---
    # Shows market regimes, volatility clustering, and stability
    plt.figure(figsize=(16, 6))
    # Using 'date_id' as a proxy for time index
    df_eda.set_index('date_id')[TARGET_COL].tail(2000).plot(linewidth=0.5) 
    plt.title('2. Time Series of Excess Returns (Last 2000 Days)')
    plt.ylabel(TARGET_COL)
    plt.show()

    # --- CHART 3: Top Correlated Features ---
    # Identifies the features most related to the target variable
    correlation_data = df_eda.corr(method='spearman')[TARGET_COL].abs().sort_values(ascending=False).head(11)
    # Remove the target column itself from the list
    correlation_data = correlation_data.drop(TARGET_COL, errors='ignore') 
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x=correlation_data.index, y=correlation_data.values)
    plt.title('3. Top 10 Features by Absolute Spearman Correlation with Target')
    plt.ylabel('Absolute Spearman Correlation')
    plt.show()
    
except Exception as e:
    print(f"EDA ERROR: Could not complete EDA visuals. {e}")

# Clean up the dataframe to save memory before model training starts
del df_eda 

print("--- EDA Complete. Proceeding to Model Training ---")

I'll update the table, adding the clarification that the aggressive tuning for $\mathbf{R^2}$ was reversed to implement necessary regularization, which now means **overfitting is under control**.

---

## 🏆 Final MVP Hyperparameter Rationale (Generalization Focus)

| Hyperparameter | Value | Role in the Strategy |
| :--- | :--- | :--- |
| **$\mathbf{n\_estimators}$** | **150** | **Safety/Execution MVP:** This is the most critical parameter, as its low value ensures the training time stays well under the $\mathbf{900\text{-second}}$ limit, preventing submission failure. **This parameter remains a non-negotiable safety lock.** |
| **$\mathbf{\text{learning\_rate}}$** | **0.07** | **Generalization MVP:** This value was dialed back from aggressive levels (0.18) to **control overfitting**. It introduces necessary regularization (bias) to prevent the model from memorizing noise, maximizing performance on the **unseen private test set**. |
| **$\mathbf{\text{max\_depth}}$** | **9** | **Complexity MVP:** This value was reverted from 10 to reduce model complexity, further **controlling overfitting**. It allows each tree to capture robust, generalized feature interactions efficiently. |

This final configuration ensures the model is both **compliant** (it runs and is scored) and **generalized** (overfitting is controlled, maximizing the chance of a high score on future data).

In [ ]:
# HULL TACTICAL - FINAL v21: LGBM OPTIMIZATION (GENERALIZATION & STABILITY)

import os
import pandas as pd
import numpy as np
import polars as pl
import joblib 
import lightgbm as lgb
from pathlib import Path
from kaggle_evaluation.default_inference_server import DefaultInferenceServer
from sklearn.metrics import r2_score, mean_squared_error
import warnings
import logging

# --- TENSORFLOW/WARNING SUPPRESSION ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  
logging.getLogger('tensorflow').setLevel(logging.FATAL)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
# ------------------------------------------------------------------------------------------------

# --- 🎯 CUSTOMIZABLE CONFIGURATION ---
KAGGLE_INPUT_PATH = '/kaggle/input/hull-tactical-market-prediction/'
#MODEL_SAVE_PATH = '/kaggle/working/lgbm_ensemble_model.pkl' 
MODEL_SAVE_PATH = '/tmp/lgbm_ensemble_model.pkl'
TARGET_COL = 'market_forward_excess_returns'

# 1. TIME-CONSTRAINT PARAMETER (MAX SAFE VALUE)
N_ESTIMATORS = 150 
# 2. TRAINING DATA WINDOW SIZE
TRAINING_WINDOW_SIZE = 6000
# 3. SHARPE-OPTIMIZED SCALING FACTOR 
SCALING_FACTOR = 0.55 
# 4. PERFORMANCE TUNING: REVERTED TO STABLE/GENERALIZED VALUE
LEARNING_RATE = 0.7 
# 5. PERFORMANCE TUNING: REVERTED TO STABLE/GENERALIZED VALUE
MAX_DEPTH = 9  
# 6. VOLATILITY FEATURE NAME
VOLATILITY_FEATURE = 'VOL_20D'
# --------------------------------------

# List all possible feature columns 
ALL_POSSIBLE_FEATURE_NAMES = [
    'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20',
    'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15',
    'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 
    'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10', 'M11', 'M12', 'M13', 'M14', 'M15', 'M16', 'M17', 'M18', 'M19', 'M20',
    'I1', 'I2', 'I3', 'I4', 'I5', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 
    'MOM1', 'MOM2', 'MOM3', 'MOM4', 'MOM5', 'MOM6', 'MOM7', 'MOM8', 'MOM9', 'MOM10', 
    'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15',
    VOLATILITY_FEATURE 
]
EXCLUDE_COLS = [
    'date_id', TARGET_COL, 'forward_returns', 'risk_free_rate', 'lagged_forward_returns', 'is_scored' 
]
GLOBAL_TRAINING_FEATURE_COLS = []

# --- GLOBAL INFERENCE STATE ---
GLOBAL_MODEL = None 
GLOBAL_INFERENCE_FEATURE_COLS = [] 
MODEL_LOADED = False
PREDICTION_COUNT = 0


# --- TRAINING PHASE (LGBM Component) ---
print("--- PHASE I: MODEL TRAINING (LGBM Component) ---")
try:
    df_train = pd.read_csv(Path(KAGGLE_INPUT_PATH) / 'train.csv')
    
    # --- VOLATILITY FEATURE ENGINEERING ---
    df_train[VOLATILITY_FEATURE] = df_train[TARGET_COL].rolling(window=20).std()
    # --------------------------------------

    current_feature_cols = [col for col in ALL_POSSIBLE_FEATURE_NAMES if col in df_train.columns and col not in EXCLUDE_COLS]
    GLOBAL_TRAINING_FEATURE_COLS = current_feature_cols
    
    for col in GLOBAL_TRAINING_FEATURE_COLS:
        df_train[col] = pd.to_numeric(df_train[col], errors='coerce')
    
    df_train.dropna(subset=[TARGET_COL, VOLATILITY_FEATURE], inplace=True) 
    df_train = df_train.tail(TRAINING_WINDOW_SIZE).copy() 

    X = df_train[GLOBAL_TRAINING_FEATURE_COLS]
    y = df_train[TARGET_COL]

    # --- HYPERPARAMETER ADJUSTMENT FOR GENERALIZATION (WFO Principle) ---
    lgbm = lgb.LGBMRegressor(
        objective='regression', 
        metric='rmse', 
        n_estimators=N_ESTIMATORS,        
        learning_rate=LEARNING_RATE,      # GENERALIZATION VALUE
        max_depth=MAX_DEPTH,             # GENERALIZATION VALUE
        n_jobs=-1, 
        random_state=42
    )
    
    lgbm.set_params(verbose=-1) 
    print(f"Training LightGBM model on {len(X.columns)} features...") 
    
    # --- FIT WITHOUT EARLY STOPPING ---
    lgbm.fit(X, y) 
    # ----------------------------------
    
    # --- METRIC CALCULATION AND DISPLAY (Use full data for final report) ---
    y_pred = lgbm.predict(X) 
    train_rmse = np.sqrt(mean_squared_error(y, y_pred))
    train_r2 = r2_score(y, y_pred)
    print(f"   LGBM Training RMSE (Full Data): {train_rmse:.4f}")
    print(f"   LGBM Training R² (Full Data): {train_r2:.4f}")
    # ----------------------------------------------------------------------

    joblib.dump({
        'model': lgbm, 
        'features': GLOBAL_TRAINING_FEATURE_COLS 
    }, MODEL_SAVE_PATH)
    print(f"Trained LGBM model saved to: {MODEL_SAVE_PATH}")

except Exception as e:
    print(f"FATAL ERROR during TRAINING PHASE: {e}")
    # Save a safe fallback model if training fails
    class DummyModel:
        def predict(self, X): 
            return np.array([0.0] * len(X))
            
    joblib.dump({'model': DummyModel(), 'features': []}, MODEL_SAVE_PATH)
    
# --- END TRAINING PHASE ---


# --- INFERENCE PHASE (LGBM-ONLY) ---

print("\n--- PHASE II: SUBMISSION INFERENCE SETUP (LGBM-ONLY FAILSATE) ---")

def predict(test: pl.DataFrame) -> float:
    
    global MODEL_LOADED, GLOBAL_MODEL, GLOBAL_INFERENCE_FEATURE_COLS, PREDICTION_COUNT
    
    if not MODEL_LOADED:
        try:
            loaded_lgbm = joblib.load(MODEL_SAVE_PATH) 
            GLOBAL_MODEL = loaded_lgbm['model']
            GLOBAL_INFERENCE_FEATURE_COLS = loaded_lgbm['features']
            
            MODEL_LOADED = True
        except Exception as e:
            print(f"FATAL ERROR during model loading: {e}. Returning constant 1.0 (Safe Failsafe).")
            return 1.0 

    # --- INFERENCE DATA PREP ---
    df_test = test.to_pandas().iloc[0:1].copy()
    
    # 1. Prepare LGBM Input (Point-in-time)
    X_lgbm = df_test.reindex(columns=GLOBAL_INFERENCE_FEATURE_COLS, fill_value=np.nan)
    for col in X_lgbm.columns:
        X_lgbm[col] = pd.to_numeric(X_lgbm[col], errors='coerce').fillna(0.0) 

    # --- PREDICTION (LGBM-ONLY) ---
    
    if X_lgbm.shape[1] > 0:
        pred_lgbm = GLOBAL_MODEL.predict(X_lgbm)[0]
    else:
        pred_lgbm = 0.0

    # Component C: Simple Linear Prediction (0.2 Weight)
    pred_linear = df_test['M1'].iloc[0] * 0.0001 if 'M1' in df_test.columns and not pd.isna(df_test['M1'].iloc[0]) else 0.0
    
    # 4. Blending (Weighted Average: 0.8 LGBM, 0.0 LSTM, 0.2 Linear)
    WEIGHTS = [0.8, 0.0, 0.2] 
    raw_prediction = (pred_lgbm * WEIGHTS[0]) + (0.0 * WEIGHTS[1]) + (pred_linear * WEIGHTS[2])

    
    # --- Allocation Sizing (Using SCALING_FACTOR = 0.55) ---
    predicted_allocation = 1.0 + (raw_prediction * SCALING_FACTOR)
    final_allocation = np.clip(predicted_allocation, 0.0, 2.0)

    PREDICTION_COUNT += 1
    if PREDICTION_COUNT % 50 == 0:
        print(f"--- LGBM Progress: Step {PREDICTION_COUNT} ---")

    return float(final_allocation)

# --- 3. RUN THE INFERENCE SERVER ---

inference_server = DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("Running in RERUN mode...")
    inference_server.serve()
else:
    print("Running local gateway for testing...")
    inference_server.run_local_gateway((KAGGLE_INPUT_PATH,))

LR 0.07 SCORE 0.484

LR 0.18 SCORE 0.502 

LR 0.58 SCORE 0.527 LGBM Training R² (Full Data): 0.9908

LR 0.7 SCORE        LGBM Training R² (Full Data): 0.9952